До нейронок для распознания изображений юзали Histogram of oriented gradients

### Решение задачи без CNN:

Решение задачи классификации на MNIST при помощи обычной полносвязной нейронной сети:

Разворачиваем картинку в вектор 1x1024, то есть на вход будем принимать вектор из 1024 булевых значений, вот так прогоним через несколько слоев и в конце получим финальные логиты из которых получим вероятности принадлежности при помощи софтмакс

Недостатки:
- в первом слое слишком много нейронов, то есть слишком много обучаемых параметров, поэтому сильно переобучается
- ломаются пространственные отношения

Когда человек смотрит на картинку, он распознает паттерны картинки

### Свертка:

- У ЧБ картинки просто яркость от 0 до 255 делаем, про RGB ниже

* Фильтр(ядро) - матрица NxN, которую мы применяем для свертки, мы поочередно накладываем фильтр на пиксели изображения и поэлементно перемножаем элементы фильтра на кусочек изображения NxN, а затем суммируем все эти произведения - по сути мы применяем то самое поэлементное тензорное умножение для ядра и всех подматриц NxN из матрицы изображения.

- Если была картинка 32на32, то станет 30на30 после свертки (уменьшение размера зависит от размера ядра свертки). Размер картинки изменяется на (N-1)x)(N-1) относительно его размера до свертки, при stride = 1. (если на ядре 3на3 рассматривать, то мы как бы не можем уже залезть на крайние 2 столбца/строки, поэтому размер уменьшается)

- Картинка после свертки - карта активации

#### Padding

Используется для того, чтобы после свертки картинка не меняла свой размер - мы обрамляем исходную картинку квадратом из нулей и получим 34на34 и она перейдет в 32на32

#### Stride

Насколько мы смещаемся при поочередном накладывании фильтра на изображение

#### Свертка цветных картинок:

Цветная картинка имеет суть как 3 ЧБ картинки - она представляется тензором размера Высота x Длина x 3 цветовых канала, внутри канала циферки тоже обозначают яркость данного цвета. Короче цветная картинка - это просто 3 двумерные картинки, а потом уже как хотим их сворачиваем

1) 3d свертка - тензор 3 ранга также поэлементно умножаем на фильтр - тензор 3 ранга и получаем двумерную карту активации
2) 2d свертка по каждому каналу отдельно - получаем 3 карты активации для этого одного ядра - затем можно их выпрямить в векторы и получить уже вектор признаков для этого ядра либо подавать на следующий сверточный слой

#### Зачем нужны фильтры?

- Фильтры реагируют на паттерны на изображении. Для примера : если выбрать фильтр, реагирующий на прямые линии, то при применении к изображению четверки, мы получим большие цифры на карте активации.

- Карты активации показывают, насколько сильно фильтр активировался на картинку, поэтому они и называются картами активации.

- Если мы применяем фильтр, который реагирует на какие-то паттерны и мы видим, что на карте активации большие числа, значит активация на фильтр сильная и там присутствуют паттерны, под которые подобран фильтр

- Карта активации - это не то, что сохраняет какие-то части исходного изображения, все что она делает - показывает реакцию на фильтр, если на ней большие по модулю числа, то активация сильная, если маленькие - слабая (фильтр не сохраняет части исходного изображения, он работает как лампочка - загорается, когда паттерны подходящие под фильтр есть на изображении)

Допустим мы получили две карты активации. Затем мы каждую из них flatten-им, потом эти векторы конкатенируем в один вектор и получаем входной слой. То есть при помощи карт активации мы получаем признаковое представление картинки

То есть как в итоге выглядит одна свертка:
- Мы выбираем количество ядер свертки, их размеры и структуру
- проходимся этими ядрами с установленным stride-ом по изображению - получаем карту активации для каждого ядра
- flatten-им все карты активации
- конкатенируем это все в один вектор

#### В итоге как cnn выглядит:

1) Подаем картинку
2) Применяем несколько сверток
3) Флаттеним полученные карты активации после сверточных слоев - получаем вектор, который олицетворяет нашу картинку
4) Используем полученный вектор на входе в полносвязную сеть
5) Полносвязная сеть выдает нам логиты
6) Оцениваем модель на полученных логитах

- Выпрямление после свертки означает, что мы закончили свертку и будем дальше передавать в FC слои полученый вектор признаков

Так ну и что, большие или маленькие числа на картах активации, как это нам поможет классифицировать картинки??? А очень просто: то что мы получаем на выходе из сверточного слоя можно мыслить как признаковое описание конкретной картинки, поэтому большие/мальенкие значения на карте активации соответствуют большому/маленькому значению признака в выходном векторе. Ну а тут уже все понятно, грубо говоря, у кого похожие признаки, те и похожи, а похожесть уже определяется минимумом лосс функции

#### Несколько слоев сверток

При помощи нескольких слоев свертки можно уже работать с довольно сложными изображениями:
- Каждый следующий слой свертки получает в качестве входов карты активации с предыдущего

Пример: картинка 32на32 (рассмотрим один канал от этой картинки), все сворачиваем со stride=1

- На 1 сверточном слое используем 5 ядер 3на3 --> получим 5 карт активации 30на30 (то есть имеем тензор 30на30на5)
- Теперь на полученном тензоре используем 3 ядра свертки размера 3на3на5 --> получаем 3 карты активации размером 28на28 (то есть имеем тензор 28на28на3 после этого слоя)

- Теперь можно было бы еще один сверточный слой сделать с ядрами размера NнаNна3 и получили бы тензор AxAxN и так можно до бесконечности сворачивать.

##### Примеры результата прогона по 4 слоям свертки:

Наша свертка состоит из 4 слоев и мы распознаем картинки с лицами людей, что в итоге будет получаться:

1) Если подавать на вход низкоуровневые паттерны(линии различные), то фильтры 1 слоя сверток будут содержать довольно большие числа - они активируются
2) Если подавать изображения носа/рта/губ , то фильтры 2-го слоя будут наиболее четко активироваться
3) И на последнем слое у нас будут карты активации содержать наибольшие числа, если мы подаем целую картинку лица, а если, например, подать туда картинку чайника, то там будут маленькие по модулю числа на картах активации
4) Последний слой содержит самую высокоуровневую информацию, мы просто берем все карты активации с последнего слоя и растягиваем их и конкатенируем и получаем вектор признаков для данной картинки, который в FC сеть передаем

5) Получаем в конце FC конечные логиты и на них применяем softmax, затем считаем cross-entropy loss

#### О кросс-энтропии

На логитах FC слоя мы используем softmax (argmax - зло) и получаем вектор из вероятностей принадлежности к каждому классу, затем мы хотим ввести какую-то функцию потерь для того, чтобы понимать в какую сторону крутить веса, чтобы все было норм. А все норм тогда, когда у нас все вероятность для нужного класса близка к 1. Итак введем Cross-entropy loss-function для классификации на M классов:
$$
l = -\sum_{c=1}^M y_{c}log(p_c)
$$
Здесь $y_c$ - 1 или 0 - принадлежность данного объекта размеченной выборки к классу под номером $c$, $p_c$ - наш $c$-ый выход софтмакса - предсказание вероятности для $c$ класса

Cross-Entropy loss для данного батча - это сумма cross-entropy для каждого объекта из батча - обозначим это за L. Наша задача найти $\nabla_{W_i} L$ - по всем i слоям FC сети - для этого мы просто юзаем бэкпроп - поочередно дифференцируем, ну и получаем че надо, затем по всех слоях обновляем все весы.

#### Функции активации на свертке

- После каждого слоя свертки надо применять функцию активации
- Просто к каждому элементу каждой карты активации применяем функцию активации

### Обучаемые параметры сверточных слоев

**У FC слоев в CNN все ясно. А где обучаемые параметры у сверточных слоев??? Все ядра -- обучаемые параметры!!!!!!**

- В процессе обучения с учителем нейронная сеть должна сама подобрать ядра

- **Все ядра обучаемые!!!**

- В процессе обучения фильтры следующих слоев подстративаются под предыдущие

Свертка - это просто линейная функция, поэтому конечно же по параметрам ядер можно искать производные функции потерь и подбирать их

Элементы ядер точно так же бэкпропом подбираются

## Pooling

пуллинг - уменьшение размерностей карт активации

Зачем нужен пуллинг:

- уменьшение размерности карт активации
- уменьшение чувствительности к положению объектов на картинке (это полезно, так как если надо классифицировать цифру, то нам бы хотелось ее определить вне зависимости в центре она или сдвинута), понятно, что не на всех фотографиях кошки и собаки прямо в середине

- Чем больше stride - тем меньше будет карта активации, также padding позволяет наоборот делать картинку больше

**Зачем уменьшать размерность:**

- Уменьшение размерности карт активации нужно для того, чтобы было меньше весов в FC части и все вычислялось быстрее

- Уменьшить число параметров в картах активации

Как оно работает:
Делим карту активации на квадратики NxN и из каждого квадратика берем max или average значения и получаем уменьшенную новую карту активации (чем больше N, тем больше информации мы потеряем)

**Пулинг слой:**

После получения карт актвиации из картинки (с предыдущего слоя) применяем пуллинг к каждой карте активации и получаем новые уменьшенные карты активации, которые потом передаем дальше/ выпрямляем и конкатенируем и передаем в FC

**Кратко про AlexNet:**

Input - CL1 - CL2 - CL3 -CL4 -CL5 - FCL6- FCL7 - FCL8

CL - это conv + pool + ReLU

При больших количествах карт активации лучше мыслить свертку как 3d процедуру

- Input: 227x227x3 (трехканальная картинка)
  
- CL1: 55x55x96 (получаем 96 карт активации размером 55x55) - можно либо 3d свертку юзать 96-ю разными трехмерными ядрами, либо можно поделить на 3 канала и каждый канал 32 ядрами свернуть и получить по 32 карты на каждый канал и соединить их снова в один тензор
  
- CL2: 27x27x256 - здесь мы получаем 256 карт активации, сворачивая 3d ядрами 256 раз(можно мыслить, что мы двигаем параллелипипед по большому параллелипипеду) или можно мыслить, что мы к каждой карте активации из входных 96 применяем некоторое количество ядер и в сумме применяем 256 ядер, то есть 256 карт активации
  
- CL3: 13x13x384 - свернули 384 разными 3d ядрами - получили 384 карты активации

- CL4: 13x13x384 - свернули 384 разными 3d ядрами размера 1x1x384 или может можно мыслить так: применили по одной свертке ядрами 1x1 на каждой входной карте активации и получили другие 384 карты активации того же размера

- CL5: 13x13x256 - применяем свертки 1х1x? на картах активации, где ? зависит от stride

- Теперь этот выход пуллится, флэттенится и конкатенируются векторы - получаем вектор длины 4096 - его подаем в FC сетку

### Задачи CV


1) Классификация
2) Detection - по картинке нужно понять, есть ли на картинке определенные объекты, которые нам нужны и обвести их, если они есть (детекция объектов для беспилотных автомобилей, вообще у беспилотников надо вообще и сегментировать кучу всего и там вообще куча задач...)
3) Segmentation - покрасить все пиксели, принадлежащие нужному нам объекту в определенный цвет (пример: сегментация опухали и здоровых тканей по снимку мозга)

Еще задачи:

- поиск по изображениям
- оценка положения/расстояния
- OCR (optical character recogniton) - автоматический перевод/ улучшение качества фотографий/документов/скан чеков/визиток

**Video Understanding** - задача computer vision:

- image description (на стыке с NLP), например, описание того, что происходит на фотографии -- выделение в баундин боксы отдельных объектов и их описание
- subtitles generation
- action description -- описание того, что происходит на видео/картинке
- **pose estimation** -- например, по видео, определять где находятся ключевые точки человека и в реальном времени по видео понимать как движутся эти ключевые точки

GAN (генеративные модели)

Если совместить GAN и Pose estimation, то можно перенести движения одного человека на другого человека